In [ ]:
import pandas as pd
dataset=pd.read_csv('../input/insurance/insurance.csv')
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset.skew()

In [ ]:
dataset.isnull().sum().sort_values(ascending=False)

In [ ]:
!pip install pandas-profiling

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(dataset, title="EDA Report")
profile

In [ ]:
dataset=dataset.drop_duplicates()
dataset.head()

In [ ]:
data = dataset.sample(frac=0.99, random_state=42)
data_unseen = dataset.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(data.shape))

In [ ]:
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
!pip install pycaret

In [ ]:
from pycaret.regression import * 

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
categorical = []
for i in data.columns:
    if (data[i].dtype=='object'):
        categorical.append(i)
print("Categorical Attribute : {}\n ".format(len(categorical)))
for x in range(len(categorical)): 
    print(categorical[x])

In [ ]:
data[categorical].nunique()

In [ ]:
insurance = setup(data = data,
                  target = 'charges',
                  session_id=1,
                  train_size = 0.8,
                  transform_target = True,
                  normalize = True, #rescale the values of numeric columns
                  handle_unknown_categorical = True, 
                  unknown_categorical_method = 'most_frequent',
                  remove_multicollinearity = True, #rop one of the two features that are highly correlated with each other
                  ignore_low_variance = True,#all categorical features with statistically insignificant variances are removed from the dataset.
                  combine_rare_levels = True,
                  normalize_method='robust',
                 categorical_features=['sex','smoker', 'region'], #categorical features
                  numeric_features=['age',  'bmi', 'children'])

In [ ]:
best=compare_models()

In [ ]:
rf= create_model('rf')

In [ ]:
print(rf)

In [ ]:
tuned_rf = tune_model(rf,optimize = 'MAE')

In [ ]:
evaluate_model(tuned_rf) #Graphical plot 

In [ ]:
predict_model(tuned_rf)# Test data evaluation 

In [ ]:
final_rf = finalize_model(tuned_rf) # Final model 
final_rf

In [ ]:
predict_model(final_rf)#final test model evaluation 

In [ ]:
unseen_predictions = predict_model(final_rf, data=data_unseen)#evaluation on unseen data
unseen_predictions.head()

In [ ]:
from pycaret.utils import check_metric
check_metric(unseen_predictions.charges, unseen_predictions.Label, 'MAE')